In [1]:
import os
# These are the default AWS configurations for the Analysis Sandbox.
# that are set in the environmnet variables. 
aws_default_config = {
    #'AWS_NO_SIGN_REQUEST': 'YES', 
    'AWS_SECRET_ACCESS_KEY': 'fake',
    'AWS_ACCESS_KEY_ID': 'fake',
}

# To access public bucket, need to remove the AWS credentials in 
# the environment variables or the following error will occur.
# PermissionError: The AWS Access Key Id you provided does not exist in our records.

for key in aws_default_config.keys():
    if key in os.environ:
        del os.environ[key]

In [2]:
import logging
import os
import queue
from threading import Thread

import click
import datacube
import fsspec
from odc.dscache import create_cache
from odc.dscache.apps.slurpy import EOS, qmap
from odc.dscache.tools import (
    bin_dataset_stream,
    dataset_count,
    db_connect,
    dictionary_from_product_list,
    mk_raw2ds,
    ordered_dss,
    raw_dataset_stream,
)
from odc.dscache.tools.tiling import parse_gridspec_with_name
from odc.stats.model import DateTimeRange
from tqdm import tqdm

from deafrica_conflux.cli.logs import logging_setup
from deafrica_conflux.hopper import bin_solar_day, persist
from deafrica_conflux.io import (
    check_dir_exists,
    check_file_exists,
    check_if_s3_uri,
    find_geotiff_files,
)
from deafrica_conflux.text import parse_tile_ids

In [3]:
verbose = 1
# Grid name africa_{10|20|30|60}
grid_name = "africa_30"
# Datacube product to search datasets for.
product = "wofs_ls"
# Only extract datasets for a given time range," "Example '2020-05--P1M' month of May 2020
temporal_range = "2023-03--P3M"
# Compression setting for zstandard 1-fast, 9+ good but slow
complevel = 6
# Path to the directory containing the polygons raster files.
polygons_rasters_directory = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters"
# Regular expression for filename matching when searching for the polygons raster files.
pattern = ".*"
# Overwrite existing cache file.
overwrite = True
# Directory to write the cache file to.
output_directory = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/"

In [4]:
# Set up logger.
logging_setup(verbose)
_log = logging.getLogger(__name__)

In [5]:
# Support pathlib Paths.
polygons_rasters_directory = str(polygons_rasters_directory)
output_directory = str(output_directory)

In [6]:
# Create the output directory if it does not exist.
is_s3 = check_if_s3_uri(output_directory)
if is_s3:
    fs = fsspec.filesystem("s3")
else:
    fs = fsspec.filesystem("file")

if not check_dir_exists(output_directory):
    fs.makedirs(output_directory, exist_ok=True)
    _log.info(f"Created directory {output_directory}")

[2023-12-04 13:12:53,642] {credentials.py:611} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [7]:
# Validate the product
products = [product]
# Connect to the datacube.
dc = datacube.Datacube()
# Get all products.
all_products = {p.name: p for p in dc.index.products.get_all()}
if len(products) == 0:
    raise ValueError("Have to supply at least one product")
else:
    for p in products:
        if p not in all_products:
            raise ValueError(f"No such product found: {p}")

In [8]:
# Parse the temporal range.
temporal_range_ = DateTimeRange(temporal_range)

output_db_fn = f"{product}_{temporal_range_.short}.db"
output_db_fp = os.path.join(output_directory, output_db_fn)

# Check if the output file exists.
if check_file_exists(output_db_fp):
    if overwrite:
        fs.delete(output_db_fp, recursive=True)
        _log.info(f"Deleted {output_db_fp}")
        # Delete the local file created before uploading to s3.
        if is_s3:
            if check_file_exists(output_db_fn):
                fsspec.filesystem("file").delete(output_db_fn)
                _log.info(f"Deleted local file created before uploading to s3 {output_db_fn}")
    else:
        raise FileExistsError(f"{output_db_fp} exists!")

[2023-12-04 13:12:54,194] {1838406043.py:11} INFO - Deleted s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/wofs_ls_2023-03--P3M.db


In [9]:
# Create the query to find the datasets.
query = {"time": (temporal_range_.start, temporal_range_.end)}
_log.info(f"Query: {query}")

[2023-12-04 13:12:54,199] {2478973304.py:3} INFO - Query: {'time': (datetime.datetime(2023, 3, 1, 0, 0), datetime.datetime(2023, 5, 31, 23, 59, 59, 999999))}


In [10]:
_log.info("Getting dataset counts")
counts = {p: dataset_count(dc.index, product=p, **query) for p in products}

n_total = 0
for p, c in counts.items():
    _log.info(f"..{p}: {c:8,d}")
    n_total += c

if n_total == 0:
    raise ValueError("No datasets found")

[2023-12-04 13:12:54,205] {279200897.py:1} INFO - Getting dataset counts
[2023-12-04 13:12:54,263] {279200897.py:6} INFO - ..wofs_ls:   22,852


/usr/local/lib/python3.10/dist-packages/datacube/drivers/postgres/_api.py:752: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  select(


In [11]:
_log.info("Training compression dictionary...")
zdict = dictionary_from_product_list(dc, products, samples_per_product=50, query=query)
_log.info("Done")

[2023-12-04 13:12:54,268] {2806460463.py:1} INFO - Training compression dictionary...
[2023-12-04 13:12:54,479] {2806460463.py:3} INFO - Done


In [12]:
if is_s3:
    cache = create_cache(output_db_fn, zdict=zdict, complevel=complevel, truncate=True)
else:
    cache = create_cache(output_db_fp, zdict=zdict, complevel=complevel, truncate=True)

In [13]:
raw2ds = mk_raw2ds(all_products)

def db_task(products, conn, q):
    for p in products:
        if len(query) == 0:
            dss = map(raw2ds, raw_dataset_stream(p, conn))
        else:
            dss = ordered_dss(dc, product=p, **query)

        for ds in dss:
            q.put(ds)
    q.put(EOS)

conn = db_connect()
q = queue.Queue(maxsize=10_000)
db_thread = Thread(target=db_task, args=(products, conn, q))
db_thread.start()

In [14]:
dss = qmap(lambda ds: ds, q, eos_marker=EOS)
dss = cache.tee(dss)

In [15]:
cells = {}
grid, gridspec = parse_gridspec_with_name(grid_name)
cache.add_grid(gridspec, grid)

cfg = dict(grid=grid)
cache.append_info_dict("stats/", dict(config=cfg))

dss = bin_dataset_stream(gridspec, dss, cells, persist=persist)

label = f"Processing {n_total:8,d} {product} datasets"
with tqdm(dss, desc=label, total=n_total) as dss:
    for _ in dss:
        pass

Processing   22,852 wofs_ls datasets: 100%|██████████| 22852/22852 [00:35<00:00, 639.99it/s]


In [16]:
# Find the required tiles.
_log.info(f"Total bins: {len(cells):d}")
_log.info("Filtering bins by required tiles...")
geotiff_files = find_geotiff_files(path=polygons_rasters_directory, pattern=pattern)

tiles_ids = [parse_tile_ids(file) for file in geotiff_files]
_log.info(f"Found {len(tiles_ids)} tiles.")
_log.debug(f"Tile ids: {tiles_ids}")

# Filter cells by tile ids.
cells = {k: v for k, v in cells.items() if k in tiles_ids}
_log.info(f"Total bins: {len(cells):d}")

[2023-12-04 13:13:30,265] {250296155.py:2} INFO - Total bins: 4511
[2023-12-04 13:13:30,266] {250296155.py:3} INFO - Filtering bins by required tiles...
[2023-12-04 13:13:30,394] {io.py:460} INFO - Found 60 GeoTIFF files.
[2023-12-04 13:13:30,395] {250296155.py:7} INFO - Found 60 tiles.
[2023-12-04 13:13:30,419] {250296155.py:12} INFO - Total bins: 60


In [17]:
tasks = bin_solar_day(cells)

# Remove duplicate source uuids.
# Duplicates occur when queried datasets are captured around UTC midnight
# and around weekly boundary
tasks = {k: set(dss) for k, dss in tasks.items()}

tasks_uuid = {k: [ds.id for ds in dss] for k, dss in tasks.items()}

all_ids = set()
for k, dss in tasks_uuid.items():
    all_ids.update(dss)
_log.info(f"Total of {len(all_ids):,d} unique dataset IDs after filtering")

label = f"Saving {len(tasks)} tasks to disk"
with tqdm(tasks_uuid.items(), desc=label, total=len(tasks_uuid)) as groups:
    for group in groups:
        cache.add_grid_tile(grid, group[0], group[1])

db_thread.join()
cache.close()

[2023-12-04 13:13:30,436] {2957611989.py:13} INFO - Total of 572 unique dataset IDs after filtering


Saving 1715 tasks to disk: 100%|██████████| 1715/1715 [00:02<00:00, 599.19it/s]


In [18]:
if is_s3:
    fs.upload(output_db_fn, output_db_fp, recursive=False)
    fsspec.filesystem("file").delete(output_db_fn)

_log.info(f"Cache file written to {output_db_fp}")

# pylint:disable=too-many-locals
csv_path = os.path.join(output_directory, f"{product}_{temporal_range}.csv")
_log.info(f"Writing summary to {csv_path}")
with fs.open(csv_path, "wt", encoding="utf8") as f:
    f.write('"T","X","Y","datasets","days"\n')
    for p, x, y in sorted(tasks):
        dss = tasks[(p, x, y)]
        n_dss = len(dss)
        n_days = len(set(ds.time.date() for ds in dss))
        line = f'"{p}", {x:+05d}, {y:+05d}, {n_dss:4d}, {n_days:4d}\n'
        f.write(line)

[2023-12-04 13:13:33,586] {2328381222.py:5} INFO - Cache file written to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/wofs_ls_2023-03--P3M.db
[2023-12-04 13:13:33,587] {2328381222.py:9} INFO - Writing summary to s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/wofs_ls_2023-03--P3M.csv
